In [ ]:
!pip install anthropic

In [ ]:
!conda create -n codex python=3.7
! conda activate codex
! git clone https://github.com/openai/human-eval
! pip install -e human-eval
# Set the working directory
%cd /content/human-eval
!pip install jsonlines

In [10]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="",
)
def generate_one_completion(prompt):
  message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    system="You are an intelligent programmer. You must complete the python function given to you by the user. And you must follow the format they present when giving your answer!",
    messages=[
        {"role": "user", "content":  prompt}
    ]
  )
  return(message.content[0].text)


In [13]:
import pandas as pd
import time
import jsonlines
import numpy as np

if __name__ == '__main__':
    jsonObj = pd.read_json(path_or_buf="https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz", lines=True)
    i = 1
    command = 0
    durationTimes = []
    with jsonlines.open('Claude3.jsonl', mode='w') as writer:
        with jsonlines.open('ClaudeTimer3.jsonl', mode='w') as timer:
            while i < 164:
              if command < 4:
                start_time = time.time()
                response = generate_one_completion(jsonObj["prompt"][i])
                duration = time.time() - start_time
                timer.write({jsonObj["task_id"][i]: duration})
                durationTimes.append(duration)
                response = response.replace("```python\n", "")
                response = response.replace("\n```", "")
                response = response[1:]
                completion = {'task_id': jsonObj["task_id"][i], 'completion': response}
                writer.write(completion)
                i = i + 1
                command = command + 1
              else:
                command = 0
                time.sleep(60)

    print(np.sum(duration))
    print(np.average(duration))


19.80688452720642
19.80688452720642


In [16]:
import jsonlines

def merge_jsonl_files(input_files, output_file):
    with jsonlines.open(output_file, mode='w') as writer:
        for file in input_files:
            with jsonlines.open(file) as reader:
                for line in reader:
                    writer.write(line)

# Example usage:
input_files = ['/content/human-eval/ClaudeTimer.jsonl','/content/human-eval/ClaudeTimer2.jsonl','/content/human-eval/ClaudeTimer3.jsonl']  # List of input JSONL files
output_file = 'mergedTimer.jsonl'  # Output file name

merge_jsonl_files(input_files, output_file)

In [15]:
!python3 human_eval/evaluate_functional_correctness.py merged.jsonl

Reading samples...
164it [00:00, 14395.32it/s]
Running test suites...
100% 164/164 [00:02<00:00, 69.83it/s]
Writing results to merged.jsonl_results.jsonl...
100% 164/164 [00:00<00:00, 22838.27it/s]
{'pass@1': 0.0}
